In [137]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._ 
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row


In [125]:
val karate_graph = sc.textFile("data/soc-karate/soc-karate.mtx")


karate_graph: org.apache.spark.rdd.RDD[String] = data/soc-karate/soc-karate.mtx MapPartitionsRDD[146] at textFile at <console>:77


On cherche juste à supprimer les permières lignes d'intro pour ne garder que les datas...

In [126]:
var df_karate = karate_graph.toDF.withColumn("id",monotonicallyIncreasingId)


df_karate: org.apache.spark.sql.DataFrame = [value: string, id: bigint]


In [127]:
df_karate = df_karate.withColumn("rank", row_number().over(Window.orderBy("id")))

df_karate: org.apache.spark.sql.DataFrame = [value: string, id: bigint ... 1 more field]


In [128]:
df_karate.show(30,false)

+--------------------------------------------------------------------------------+----------+----+
|value                                                                           |id        |rank|
+--------------------------------------------------------------------------------+----------+----+
|%%MatrixMarket matrix coordinate pattern symmetric                              |0         |1   |
|%-------------------------------------------------------------------------------|1         |2   |
|% UF Sparse Matrix Collection, Tim Davis                                        |2         |3   |
|% http://www.cise.ufl.edu/research/sparse/matrices/Newman/karate                |3         |4   |
|% name: Newman/karate                                                           |4         |5   |
|% [Karate club, from Wayne Zachary, 1977]                                       |5         |6   |
|% id: 2399                                                                      |6         |7   |
|% date: 1

In [129]:
df_karate = df_karate.filter(df_karate("rank")>24)

df_karate: org.apache.spark.sql.DataFrame = [value: string, id: bigint ... 1 more field]


In [130]:
df_karate = df_karate.drop("id","rank")

df_karate: org.apache.spark.sql.DataFrame = [value: string]


In [131]:
df_karate.show(5)

+-----+
|value|
+-----+
|  2 1|
|  3 1|
|  4 1|
|  5 1|
|  6 1|
+-----+
only showing top 5 rows



In [133]:
import org.apache.spark.sql.functions.split

val df_karate_splitted = df_karate.withColumn("_tmp", split($"value", "\\ ")).select(
  $"_tmp".getItem(0).as("srcId"),
  $"_tmp".getItem(1).as("dstID"),
)


import org.apache.spark.sql.functions.split
df_karate_splitted: org.apache.spark.sql.DataFrame = [srcId: string, dstID: string]


In [134]:
df_karate_splitted.show(5)

+-----+-----+
|srcId|dstID|
+-----+-----+
|    2|    1|
|    3|    1|
|    4|    1|
|    5|    1|
|    6|    1|
+-----+-----+
only showing top 5 rows



In [138]:
val rows: RDD[Row] = df_karate_splitted.rdd

rows: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[180] at rdd at <console>:92


In [153]:
val RDD_edges = df_karate_splitted.rdd
  .map(row => {
    val srcId = row.getString(0)
    val dstID = row.getString(1)
    (srcId,dstID)
  })


RDD_edges: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[211] at map at <console>:93


In [154]:
df_karate_splitted.show()

+-----+-----+
|srcId|dstID|
+-----+-----+
|    2|    1|
|    3|    1|
|    4|    1|
|    5|    1|
|    6|    1|
|    7|    1|
|    8|    1|
|    9|    1|
|   11|    1|
|   12|    1|
|   13|    1|
|   14|    1|
|   18|    1|
|   20|    1|
|   22|    1|
|   32|    1|
|    3|    2|
|    4|    2|
|    8|    2|
|   14|    2|
+-----+-----+
only showing top 20 rows



In [161]:
val dstId = df_karate_splitted.select(df_karate_splitted("dstId")).distinct
val srcId = df_karate_splitted.select(df_karate_splitted("srcId")).distinct


dstId: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [dstId: string]
srcId: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [srcId: string]


In [162]:
srcId.show(10)

+-----+
|srcId|
+-----+
|    2|
|    3|
|    4|
|    5|
|    6|
|    7|
|    8|
|    9|
|   11|
|   12|
+-----+
only showing top 10 rows



In [135]:
val RDD_vertices = sc.parallelize(df_karate_splitted)


<console>: 81: error: type mismatch;

In [140]:
rows.toDF.show()

<console>: 94: error: value toDF is not a member of org.apache.spark.rdd.RDD[org.apache.spark.sql.Row]